<a href="https://colab.research.google.com/github/SherifNabil-Eng/AzureCognitivePoCs/blob/main/Test_FormRecInvoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install azure-ai-formrecognizer==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 878.2 kB/s eta 0:00:00


In [ ]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 5.4 MB/s eta 0:00:00


In [ ]:
# import libraries
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount('/content/drive', force_remount=force_remount)

import os
import numpy as np
import pandas as pd
from datetime import date

from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

os.chdir("/content/gdrive/My Drive/Colab Notebooks/Tests")


Mounted at /content/gdrive


In [ ]:
def analyze_receipts(receipt):

    
    # set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal
    endpoint = ""
    key = ""
    
    # create your `DocumentAnalysisClient` instance and `AzureKeyCredential` variable
    
    document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    #receipts_df = pd.DataFrame ([],columns=['Item#','Item_Description','Item_Quantity','Item_total_Price','Merchant_Address','Merchant_Name','Subtotal','Total','TransactionDate','TransactionTime'])


    ###This needs to be changed to chg directory next time
    currroot=os.getcwd() 
    print("File location using os.getcwd():", os.getcwd())
    receiptpath = os.path.join(currroot, receipt)

    with open(receiptpath, "rb") as fd:
        receiptimg = fd.read()

    poller = document_analysis_client.begin_analyze_document("prebuilt-receipt", receiptimg)
    result = poller.result()
    
    #print ("results are ",result.documents )

    receipt_dic={}
    receipt_items={}

    for receiptimg in result.documents:
        for name, field in receiptimg.fields.items():
            if name == "Items":
                print("Receipt Items:")
                
                
                for idx, item in enumerate(field.value):
                    print("...Item #{}".format(idx+1))
                    #receipts_df.loc[idx,'Item#']=idx
                    receipt_items[idx+1]=idx+1
                    for item_field_name, item_field in item.value.items():
                        print("......{}: {} has confidence {}".format(
                            item_field_name, item_field.value, item_field.confidence))
                        receipt_items[item_field_name+str(idx+1)]=item_field.value
            else:
                if not bool (receipt_dic):
                  receipt_dic["Items"]= receipt_items
                  receipt_dic[name]= field.value
                  print("{}: {} has confidence {}".format(name, field.value, field.confidence))


                else:
                  receipt_dic[name]= field.value
                  print("{}: {} has confidence {}".format(name, field.value, field.confidence))

    fd.close()
    
    return receipt_dic


In [ ]:

receipts_names = ["FormRecImgs/Receipt.jpg","FormRecImgs/Laroma.jpg","FormRecImgs/Gezira.jpg","FormRecImgs/GANT.jpg","FormRecImgs/Arabic_car_Receipt_1.jpg","FormRecImgs/Arabic_car_Receipt_2.jpg","FormRecImgs/Caribu_2.jpg","FormRecImgs/hand_written_receipt.jpg"]



receipts_len = len (receipts_names)

receipts_dic={}


if __name__ == "__main__":
    for i in range (0,receipts_len,1):
        #print ("\n receipt # {} \n".format(i+1))
        receipts_dic["receipt_no_"+str(i+1)]=analyze_receipts(receipts_names[i])
    
    receipts_df = pd.DataFrame (receipts_dic)
    print ("\n receipts_dic",receipts_dic)
    

File location using os.getcwd(): /content/gdrive/MyDrive/Colab Notebooks/Tests
Receipt Items:
...Item #1
......Description: Triple B Coconut Dream 70G has confidence 0.98
......Quantity: 1.0 has confidence 0.983
......TotalPrice: 49.0 has confidence 0.985
MerchantAddress: AddressValue(house_number=None, po_box=None, road=None, city=Gate, state=None, postal_code=None, country_region=Egypt, street_address=) has confidence 0.967
MerchantName: Caribou Coffee has confidence 0.972
Subtotal: 49.0 has confidence 0.985
Total: 49.0 has confidence 0.98
TransactionDate: 2023-01-17 has confidence 0.982
TransactionTime: 08:33:00 has confidence 0.983
File location using os.getcwd(): /content/gdrive/MyDrive/Colab Notebooks/Tests
Receipt Items:
...Item #1
......Description: Forte has confidence 0.43
......Quantity: 1.0 has confidence 0.348
......TotalPrice: 39.25 has confidence 0.466
Subtotal: 39.25 has confidence 0.987
Total: 44.75 has confidence 0.988
TotalTax: 5.5 has confidence 0.987
TransactionDat

In [ ]:
#get current path
currpath=os.getcwd()
print ("current path is",currpath)

current_date = today = str (date.today())

print ("current day is ",current_date)

# Create a Pandas Excel writer using XlsxWriter

output_path = 'receipts'+current_date+'.xlsx'

writer = pd.ExcelWriter(output_path, engine='xlsxwriter')

receipts_df.to_excel(writer,sheet_name='Receipts')



# Save Data to File
writer.save()

print (writer)

writer.close()

current path is /content/gdrive/MyDrive/Colab Notebooks/Tests
current day is  2023-02-13


/usr/local/lib/python3.8/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [ ]:
Receipt_1_Item_1 ={'Item_no':1,'Description':'late','Quantity':1,'Price':49}
Receipt_1_Item_2 ={'Item_no':2,'Description':'Triple B Coconut Dream 70G','Quantity':1,'Price':100}
Receipt_1_overall ={'Merchant':'Caribou Coffee','total':149,'Address':'Heliopolis,Egypt','Transaction_Date':'2023-01-17'}
Receipt_2_Item_1 ={'Item_no':1,'Description':'G-Eclair','Quantity':4,'Price':60}
Receipt_2_Item_2 ={'Item_no':2,'Description':'G-Melf','Quantity':2,'Price':40}
Receipt_2_overall ={'Merchant':'Pastry','total':100,'Address':'Zamalek','Transaction_Date':'2023-01-19'}
Receipt_3_Item_1 ={'Item_no':1,'Description':'Description: SHIELD C-M,SCARLET','Quantity':1,'Price':1508.77}
Receipt_3_overall ={'Merchant':'GANT','total':1720,'Address':'City Center','Transaction_Date':'2023-01-23'}

In [ ]:
Receipt_1 = {'Item_1':Receipt_1_Item_1,'Item_2':Receipt_1_Item_2,'Items_overall':Receipt_1_overall}
Receipt_2 = {'Item_1':Receipt_2_Item_1,'Item_2':Receipt_2_Item_2,'Items_overall':Receipt_2_overall}
Receipt_3 = {'Item_1':Receipt_3_Item_1,'Items_overall':Receipt_3_overall}

Receipts= {'Receipt_1':Receipt_1,'Receipt_2':Receipt_2,'Receipt_3':Receipt_3}

In [ ]:
Receipts

{'Receipt_1': {'Item_1': {'Item_no': 1,
   'Description': 'late',
   'Quantity': 1,
   'Price': 49},
  'Item_2': {'Item_no': 2,
   'Description': 'Triple B Coconut Dream 70G',
   'Quantity': 1,
   'Price': 100},
  'Items_overall': {'Merchant': 'Caribou Coffee',
   'total': 149,
   'Address': 'Heliopolis,Egypt',
   'Transaction_Date': '2023-01-17'}},
 'Receipt_2': {'Item_1': {'Item_no': 1,
   'Description': 'G-Eclair',
   'Quantity': 4,
   'Price': 60},
  'Item_2': {'Item_no': 2,
   'Description': 'G-Melf',
   'Quantity': 2,
   'Price': 40},
  'Items_overall': {'Merchant': 'Pastry',
   'total': 100,
   'Address': 'Zamalek',
   'Transaction_Date': '2023-01-19'}},
 'Receipt_3': {'Item_1': {'Item_no': 1,
   'Description': 'Description: SHIELD C-M,SCARLET',
   'Quantity': 1,
   'Price': 1508.77},
  'Items_overall': {'Merchant': 'GANT',
   'total': 1720,
   'Address': 'City Center',
   'Transaction_Date': '2023-01-23'}}}